# Example of Tide Prediction For One Date Instance

- In this example, method used to predict tide is adapated from Pytides
- This implementation will only work for known NOAA gauge stations
- Harmonic Constituents data is scraped from NOAA. 

Adapted by @rjleveque from notebook of @socoyjonathan to test local pytides and new tidetools.py.

## Tide Prediction Module Functions

 - <b>retrieve_constituents</b> - retrieves harmonic constituents from NOAA gauge station
 - <code><b>fetch_noaa_tide_data()</b></code> - retrieves datetimes, water levels and tide predictions at given NOAA tide station from NOAA's API
 
 - <b>datetimes</b> - prepares a collection of datetimes from beginning to end dates if needed
 
 - <code><b>predict_tide()</b></code> - predicts tide for desired NOAA station given station ID, start date and end date for prediction 
 - <b>datum_value</b> - retrieves datum value for desired datum reference, utilized by <b>predict_tide</b> to obtain <b>MTL</b> value
 - <code><b>detide()</b></code> - detides observed water levels with predicted tide
 - <code><b>surge()</b></code> - predicts surge at NOAA gauge station provided station ID, start date, end date, and landfall date, best for a Clawpack Simulation!

In [ ]:
import matplotlib.pyplot as plt
import datetime

#from clawpack.geoclaw import tidetools

# Eventually move tidetools.py to geoclaw
# For development purposes, this is temporarily in this repo:

import os, sys
pathstr = os.path.abspath('..')
if pathstr not in sys.path:
    sys.path.insert(0,pathstr)
import tidetools

print('Using tidetools from: %s' % tidetools.__file__)

### ****  Station Information ****

Locate NOAA station ID.  NOAA gauge stations home: https://tidesandcurrents.noaa.gov/ <br>
Fill in station ID, reference datum and date instance for tide prediction!

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Date of prediction (YEAR, MTH, DAY, HR)
prediction_date = datetime.datetime(2005, 8, 29, 11)

### Tide Prediction

Prediction of tide at specified location (station ID) and specified time (GMT) implemented below by calling <code><b>predict_tide()</b></code> method with the following arguments: <b> station_id, beg_prediction_date, end_prediction_date</b>. Note: datum argument is optional

<br> 

To predict tide at an instant, set <b>beg_prediction_date</b> and <b>end_prediction_date</b> in <code><b>predict_tide()</b></code> method to the same date!

In [ ]:
#NOAA Data Scraping Implementation      
height = tidetools.predict_tide(station_id, prediction_date, prediction_date, datum='MTL')
times = tidetools.datetimes(prediction_date, prediction_date) # in meters
print(height[0], "meters")

*******************************************************************************************************************

# Example of Tide Prediction In A Date Interval 

###   Station Information 

Fill in station ID, a beginning date and an end date for tide prediction below

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Beginning and End Dates 
beg_date = datetime.datetime(2005, 8, 26, hour=0)
end_date = datetime.datetime(2005, 8, 31, hour=0)

#Predict tide with arguments set as: (station_id, beg_prediction_date, end_prediction_date)
predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date, datum='MTL')

### Tide Predictions
Plot results in a time series plot

In [ ]:
#Method datetimes() makes a range of datetimes given arguments: (beg_prediction_date, end_prediction_date)
times = tidetools.datetimes(beg_date, end_date)

plt.figure(figsize=(20,10))
plt.plot(times, predicted_tide, "-", label="Tide Prediction")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Meters'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('My Prediction for Station {}'.format(station_id))
plt.show()

*******************************************************************************************************************

# Example Comparing NOAA vs Our Tide Prediction In A Date Interval 

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Beginning and End Dates 
beg_date = datetime.datetime(2005, 8, 26)
end_date = datetime.datetime(2005, 8, 31)

#Predict Tide 
predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date, datum='MTL')

- Calling function <code><b>fetch_noaa_tide_data()</b></code> with arguments set as <b>(station_id, beg_prediction_date, end_prediction_date)</b> retrieves datetimes, water levels and tide predictions for the specified NOAA station in the date interval provided from NOAA's API
- Data is scraped in <b>Metric</b> units, <b>GMT</b> timezone, <b>MTL</b> datum and  <b>6 min</b> intervals. These arguments are optional in <code><b>fetch_noaa_tide_data()</b></code>.

<b>RJL:</b> I'd suggest breaking the line in the next cell so it fits better when viewed in a narrow browser window.  There are also a couple other places below where long lines could be split.

In [ ]:
#Retrieve NOAA Tide Data
times, NOAA_observed_water_lvl, NOAA_predicted_tide = \
       tidetools.fetch_noaa_tide_data(station_id, beg_date, end_date, datum='MTL')

In [ ]:
#Plot Comparisons
plt.figure(figsize=(20,10))
plt.plot(times, predicted_tide, "-", label="Our Tide Prediction")
plt.plot(times, NOAA_predicted_tide, "-", label="NOAA Tide Prediction")
plt.plot(times, NOAA_observed_water_lvl, "-", label="NOAA Water Level Observation")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('Comparison of Our Prediction vs NOAA prediction for Station {}'.format(station_id))
plt.show()

*******************************************************************************************************************

<b>RJL:</b> I think you need some explanation either here or at the start of this example of what you are showing, i.e. that you have chose and date and location where there was a bit storm surge (from what event?) so the reader knows what they are looking at.  And below you might comment that the reason for detiding the observations might be to compare the surge alone with GeoClaw simulation results, for example.

# Example Detiding and Capturing A Surge for a Gauge Station 

- Calling <code><b>predict_tide()</b></code> method with arguments set as: <b>(station_id, beg_prediction_date, end_prediction_date)</b> outputs predicted tide
- Calling <code><b>fetch_noaa_tide_data()</b></code> with arguments set as <b>(station_id, beg_prediction_date, end_prediction_date)</b> retrieves datetimes, water levels and tide predictions from NOAA
- Calling <code><b>detide()</b></code> method with arguments set as: <b>(NOAA observed water level, predicted tide)</b> will output detided water level. 

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Beginning and End Dates 
beg_date = datetime.datetime(2005, 8, 26)
end_date = datetime.datetime(2005, 8, 31)

predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date)
times, NOAA_observed_water_lvl, NOAA_predicted_tide = \
       tidetools.fetch_noaa_tide_data(station_id, beg_date, end_date, datum='MTL')

surge = tidetools.detide(NOAA_observed_water_lvl, predicted_tide)

#Plot Comparisons
plt.figure(figsize=(20,10))
plt.plot(times, surge, "-", label="Our Surge Prediction")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('Detided Water Level for Station {}'.format(station_id))
plt.show()

*******************************************************************************************************************

# Example for Clawpack Storm Surge Implementation

<b>RJL:</b> It's not clear what you mean here by "storm surge implementation".  It seems like you are talking about comparing the NOAA gauges results with the surge simulated by GeoClaw, but I don't think you are showing any GeoClaw results in this notebook. This might not be clear to a reader, and when you label some curves with e.g. "our detided prediction" they might think that's the GeoClaw simulated surge, whereas I think you mean the NOAA observations minus the predicted tide that you computed from the constituents using the new `tide.py` (as opposed to the NOAA predictions downloaded from the webpage).

- Code below works best if placed in <b>gauge_afteraxes( )</b> in <b>setplot.py</b> for a storm simulation.
- Calling <code><b>surge()</b></code> method with arguments set as: <b>(station_id, beginning_date, end_date, landfall_date)</b> will output storm surge from NOAA observed water levels and predicted tide.

In [ ]:
#Station Information
station_id = '8761724'

#Beginning, End, Landfall Dates
beg_date = datetime.datetime(2005, 8, 26)
end_date = datetime.datetime(2005, 8, 31)
landfall_date = datetime.datetime(2005, 8, 29, 11, 10)

# Surge Prediction
times, surge = tidetools.surge(station_id, beg_date, end_date, landfall_date)
plt.plot(times, surge, color="b", label="Our Prediction")

*******************************************************************************************************************

# Example Iterating Through A Library Of Stations And Date Intervals

<b>RJL:</b>  Again you might add some more discussion here, saying that these particular stations and dates correspond to particular surge events (which is sort of clear from the comments such as `#katrina` in the cell below, but could be explained more, perhaps with links to NOAA pages or elsewhere describing these events?)

<b>RJL:</b> Also, in a lot of figures below it looks like the observations were pretty far from the predicted tides even before the big surge arrived, is that because the storm was already having a significant effect (and hence this deviation from predictions should be preserved in the de-tided version that you will compare to GeoClaw results, since it should also be seen in the simulations presumably) or is this because the predictions are poor for some other reason?  It would be valuable to explain this a bit more.  In the case of tsunamis, often nothing unusual happens until the first big wave arrives, but surge is probably different.

In [ ]:
station_dict = {'8761724': ('Grand Isle, LA', (2005, 8, 26), (2005, 8, 31), (2005, 8, 29, 11, 10)), #katrina
                '8760922': ('Pilots Station East, SW Pass, LA', (2005, 8, 26), (2005, 8, 31), (2005, 8, 29, 11)), #michael
                '8658120': ('Wilmington, NC', (2016, 10, 6, 12), (2016, 10, 9, 12), (2016, 10, 8, 12)), #matthew
                '8721604': ('Trident Pier, Port Canaveral, FL', (2019, 8, 24), (2019, 9, 9), (2019, 9, 4, 12)), #dorian
                '8723970': ('Vaca Key, Florida Bay, FL', (2017, 9, 6, 13), (2017, 9, 12, 13), (2017, 9, 10, 13)) #irma
               }

for (key, value) in station_dict.items():
    station_id = key
    station_name = value[0]
    beg_date = datetime.datetime(*value[1])
    end_date = datetime.datetime(*value[2])
    landfall_date = datetime.datetime(*value[3])
    
    #NOAA Data Scraping Implementation
    predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date) 
    
    times, NOAA_observed_water_lvl, NOAA_predicted_tide = \
           tidetools.fetch_noaa_tide_data(station_id, beg_date, end_date, datum='MTL')

    #Detide Water Level
    surge = tidetools.detide(NOAA_observed_water_lvl, predicted_tide)
    NOAA_surge = tidetools.detide(NOAA_observed_water_lvl, NOAA_predicted_tide)
    
    #Plot Comparisons
    plt.figure(figsize=(20,10))
    plt.plot(times, predicted_tide, "-", label="Our Tide Prediction")
    plt.plot(times, NOAA_predicted_tide, "-", label="NOAA Tide Prediction")
    plt.plot(times, NOAA_observed_water_lvl, "-", label="NOAA Water Level Observation")
    plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
    plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
    plt.title('Comparison of Our Prediction vs NOAA prediction for Station {}, {}'.format(station_id, station_name))
    plt.show()
    
    #Detided Water Level Comparison
    plt.figure(figsize=(20,10))
    plt.plot(times, surge, "-", label="Our Detided Prediction")
    plt.plot(times, NOAA_surge, "-", label="NOAA's Detided Prediction")
    plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
    plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
    plt.title('Detided Water Level Comparison of Our Prediction vs NOAA prediction for Station {}, {}'.format(station_id, station_name))
    plt.show()
    
    
    #### Clawpack Implementation (in setplot.py) ####
    times, surge = tidetools.surge(station_id, beg_date, end_date, landfall_date)
    plt.plot(times, surge, color="b", label="Our Surge Prediction")
    
    